# SI 330: Data Manipulation 
## 17 - SQL

### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

# <FONT COLOR="red">NOTE: Please install the ```pymysql``` module (conda install pymysql)

# SQL Outline

## Day 1

* AWS Overview
  * EC2, RDS

* Tasks
  * log into AWS Console
  * create EC2 instance
    * yum install mysql
    * wget .sql
    
## Day 2

* RDS
* MySQL

* Tasks
  * create RDS instance
  * load SQL file from EC2
  * browse with MySQLWorkbench
  
## Day 3

* pymysql

* Tasks
  * query goodreads
  
## Day 4

* BigQuery, Homework


## SQL

In general, we are going to:
```
SELECT some_fields FROM some_table
WHERE some_condition_exists
[ GROUP BY some_field ]
[ ORDER BY some_field ]
```

For example:
```
SELECT authors FROM books;
```
**Note:** the convention is to use UPPERCASE for SQL reserved words, and lowercase for the specifics of your instance

If we want all the fields:
```
SELECT * FROM books;
```

Adding a condition:
```
SELECT * FROM books
WHERE original_publication_year >= 2008;
```

And you can also group by a field:
```
SELECT original_publication_year,COUNT(*) FROM books
GROUP BY original_publication_year;
```

## Introduction to pymysql



```pymysql``` is a python interface to SQL servers (including MySQL, sqlite, etc.).  There are other interfaces (such as SQLAlchemy), but pymysql is the most straight-forward to work with. (Note caveats that we'll discuss in class.)

In [ ]:
import pymysql

Fill in the ```host```, ```user``` and ```password``` fields in the next code block.  The host is the RDS endpoint; user should probably be your uniqname, and password should be whatever you set you database password to.

Run the block to establish a connection to your RDS-based MySQL server:

In [ ]:
connection = pymysql.connect(host='rds_endpoint', # remove this before submitting
                             user='username', # remove this before submitting
                             password='password', # remove this before submitting
                             db='goodreads',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

Here's a "pythonic" way to query your database.  THe following is a bit 

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM books WHERE original_title = 'The Hunger Games'")
    results = cursor.fetchall()
    for result in results:
        print(result['original_title'],result['authors'])

### COUNT

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) FROM books WHERE authors = 'Suzanne Collins'")
    result = cursor.fetchone()
    print(result)

We can rename awkward column names by using the ```AS``` operator:

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("SELECT COUNT(*) AS cnt FROM books WHERE authors = 'Suzanne Collins'")
    result = cursor.fetchone()
    print(result)

### DISTINCT

Sometimes, we want to know the number of distinct (unique) values of a field.  Here are two ways of doing this:

In [ ]:
with connection.cursor() as cursor:
    cursor.execute('SELECT DISTINCT authors FROM books')
    results = cursor.fetchall()
    print(len(results))

In [ ]:
with connection.cursor() as cursor:
    cursor.execute('SELECT COUNT(DISTINCT authors) FROM books')
    result = cursor.fetchone()
    print(result)

### <font color="magenta">Q1: Which of the above two ways of finding distinct values is more efficient?  Why?</font>

## SQL from pandas

We are going to continue to use our connection object from above in this section.  Pandas plays nicely with pymysql.

In [ ]:
import pandas as pd

Just like what we found with read_csv (a relief compared to CSVDictReader), we have a read_sql method that allows us to create a dataframe based on an SQL query:

In [ ]:
df = pd.read_sql('SELECT * FROM books',con=connection)
len(df)

In [ ]:
df.head()

Let's work in pandas, and explore wildcards and regular expressions in SQL:

In [ ]:
df = pd.read_sql('SELECT * FROM books WHERE original_title = "The Hunger Games"',con=connection)
df

In SQL, ```%``` means 1 or more wildcard characters; ```?``` means exactly one wildcard character.  So, if we wanted to look for titles that contain "Hunger Games", we could use:

In [ ]:
df = pd.read_sql('SELECT * FROM books WHERE original_title LIKE "%Hunger Games%"',con=connection)
df

More powerfully, we could use regular expressions:

In [ ]:
df = pd.read_sql('SELECT * FROM books WHERE original_title REGEXP ".*Hunger Games.*"',con=connection)
df

In [ ]:
df = pd.read_sql('SELECT * FROM books WHERE original_publication_year IN (2008, 2010)',con=connection)
df.head()

Sometimes, it's nicer to wrap our SQL across multiple lines, so we can use docstrings ("""...""") to create our SQL.

Here's an example of a join using a WHERE statement:

In [ ]:
df = pd.read_sql("""
                SELECT books.original_title, ratings.rating 
                FROM books,ratings 
                WHERE books.authors LIKE '%Rowling%' 
                AND books.book_id = ratings.book_id;
                """, con = connection)   

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.sample(10)

We can leverage the groupby function in pandas to do helpful calculations:

In [ ]:
df.groupby(by='original_title')[['rating']].mean()

Here's a schematic representation of the goodreads database:

![](assets/goodreadsEER.png)

In [ ]:
df.sample(10)

### <font color="magenta">Q2: How many books in our dataset were originally published in 2016?</font>

In [ ]:
df = pd.read_sql("select count(*) from books where original_publication_year = 2017",con=connection)

In [ ]:
len(df)

In [ ]:
df

### <font color="magenta">Q3: How many books were published in each year of original publication in our dataset?</font>


In [ ]:
# insert your code here

In [ ]:
df = pd.read_sql("select original_publication_year,count(*) from books group by original_publication_year",con=connection)

In [ ]:
df.head(20)

### <font color="magenta">Q4: Do you notice anything interesting or strange about your results from the previous question?  Investigate at least one of these anomalies using either an SQL query or pandas filtering. </font>

Insert your answer here

In [ ]:
# insert your code here

### <font color="magenta">Q5: Create a dataframe that contains a list of all book titles from 2017 as well as the tag_names associated with each book.</font>

In [ ]:
# Insert your code here

In [ ]:
df = pd.read_sql("""
                SELECT books.title, tags.tag_name
                FROM books
                LEFT JOIN books_tags on books_tags.goodreads_book_id = books.goodreads_book_id
                LEFT JOIN tags on tags.tag_id = books_tags.tag_id
                                WHERE books.original_publication_year = 2017 
                """, con = connection)   

# END OF NOTEBOOK: REMEMBER TO STRIP IDETIFYING INFORMATION FROM TOP OF NOTEBOOK!
Please submit both ipynb and html formats of this notebook to Canvas.